In [3]:
import pandas as pd
from tqdm import tqdm

In [31]:
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('../../data/processed/WESAD/data_processed/30s/all_data.csv', index_col=0)
df.head(5)

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,EDA_mean,EDA_std,EDA_slope,RESP_rate,RESP_regularity,HR,IBI,RMSSD,SDNN,pNN50,lf/hf,label,subject,focus_label
0,37.715333,-35.027333,-9.694000,62.840001,5.214880,0.630601,0.026678,-0.000766,17.748666,1.755023,75.081294,847.426471,226.949936,186.465391,93.939394,1.475494,1,8,1
1,39.252000,-47.664667,7.822000,62.549339,0.544205,0.563131,0.014406,-0.000412,21.083279,1.630067,63.966173,958.669355,213.058396,133.561891,90.000000,5.933398,1,8,1
2,39.628667,-46.246667,9.764000,62.469299,1.120992,0.523506,0.009367,-0.000265,21.327375,1.591487,66.372243,911.132812,134.469912,80.095987,74.193548,44.207315,1,8,1
3,38.927333,-48.439333,6.083333,62.442314,0.428267,0.520051,0.018623,0.000048,22.168906,1.352590,78.588847,803.631757,159.088795,155.426060,58.333333,707.930928,1,8,1
4,38.919333,-47.006667,-0.914667,62.576766,1.166077,0.493109,0.007353,-0.000203,24.011434,1.809602,65.088073,940.020161,199.098718,134.709634,63.333333,6.149627,1,8,1


In [4]:
df['rulebased_label'] = -1
df.head()

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,EDA_mean,EDA_std,EDA_slope,RESP_rate,RESP_regularity,HR,IBI,RMSSD,SDNN,pNN50,lf/hf,label,subject,focus_label,rulebased_label
0,37.715333,-35.027333,-9.694000,62.840001,5.214880,0.630601,0.026678,-0.000766,17.748666,1.755023,75.081294,847.426471,226.949936,186.465391,93.939394,1.475494,1,8,1,-1
1,39.252000,-47.664667,7.822000,62.549339,0.544205,0.563131,0.014406,-0.000412,21.083279,1.630067,63.966173,958.669355,213.058396,133.561891,90.000000,5.933398,1,8,1,-1
2,39.628667,-46.246667,9.764000,62.469299,1.120992,0.523506,0.009367,-0.000265,21.327375,1.591487,66.372243,911.132812,134.469912,80.095987,74.193548,44.207315,1,8,1,-1
3,38.927333,-48.439333,6.083333,62.442314,0.428267,0.520051,0.018623,0.000048,22.168906,1.352590,78.588847,803.631757,159.088795,155.426060,58.333333,707.930928,1,8,1,-1
4,38.919333,-47.006667,-0.914667,62.576766,1.166077,0.493109,0.007353,-0.000203,24.011434,1.809602,65.088073,940.020161,199.098718,134.709634,63.333333,6.149627,1,8,1,-1


In [26]:
def detect_focus(hr, hrv, eda, resp_rate):
    HR_MIN = 60       
    HR_MAX = 100       
    HRV_MIN = 30      
    EDA_MIN = 0.05    
    EDA_MAX = 0.5     
    RESP_RATE_MIN = 12  
    RESP_RATE_MAX = 26

    hr_focus = HR_MIN <= hr <= HR_MAX
    hrv_focus = hrv >= HRV_MIN
    eda_focus = EDA_MIN <= eda <= EDA_MAX
    resp_focus = RESP_RATE_MIN <= resp_rate <= RESP_RATE_MAX 

    if hr_focus and hrv_focus and eda_focus and resp_focus:
        return 1  # Focus
    else:
        return 0  # Not focus


In [27]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    hr = float(row['HR'])
    hrv = float(row['SDNN'])
    eda = float(row['EDA_mean'])
    resp_rate = float(row['RESP_rate'])

    focus = detect_focus(hr, hrv, eda, resp_rate)
    df.at[i, 'rulebased_label'] = focus

100%|██████████| 1075/1075 [00:00<00:00, 32567.06it/s]


In [28]:
df.head()

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,EDA_mean,EDA_std,EDA_slope,RESP_rate,RESP_regularity,HR,IBI,RMSSD,SDNN,pNN50,lf/hf,label,subject,focus_label,rulebased_label
0,37.715333,-35.027333,-9.694000,62.840001,5.214880,0.630601,0.026678,-0.000766,17.748666,1.755023,75.081294,847.426471,226.949936,186.465391,93.939394,1.475494,1,8,1,0
1,39.252000,-47.664667,7.822000,62.549339,0.544205,0.563131,0.014406,-0.000412,21.083279,1.630067,63.966173,958.669355,213.058396,133.561891,90.000000,5.933398,1,8,1,0
2,39.628667,-46.246667,9.764000,62.469299,1.120992,0.523506,0.009367,-0.000265,21.327375,1.591487,66.372243,911.132812,134.469912,80.095987,74.193548,44.207315,1,8,1,0
3,38.927333,-48.439333,6.083333,62.442314,0.428267,0.520051,0.018623,0.000048,22.168906,1.352590,78.588847,803.631757,159.088795,155.426060,58.333333,707.930928,1,8,1,0
4,38.919333,-47.006667,-0.914667,62.576766,1.166077,0.493109,0.007353,-0.000203,24.011434,1.809602,65.088073,940.020161,199.098718,134.709634,63.333333,6.149627,1,8,1,1


In [29]:
df.value_counts('rulebased_label')

rulebased_label
0    697
1    378
Name: count, dtype: int64

In [30]:
df.value_counts('focus_label')

focus_label
1    595
0    480
Name: count, dtype: int64

In [33]:
print(classification_report(df['focus_label'], df['rulebased_label'],target_names=['Not Focus', 'Focus']))

              precision    recall  f1-score   support

   Not Focus       0.55      0.80      0.66       480
       Focus       0.75      0.48      0.58       595

    accuracy                           0.62      1075
   macro avg       0.65      0.64      0.62      1075
weighted avg       0.66      0.62      0.62      1075

